This is an auto-generated notebook

In [1]:
import pandas as pd
import numpy as np
import keras as ks

Using TensorFlow backend.


In [2]:
df = pd.read_csv("indexByMatch_batchUdacity_205_CLEANED_NORM.csv", index_col="Unnamed: 0")
labels = df["alphaVictory"]
df.drop("alphaVictory", axis=1, inplace=True)

In [3]:
data = df.as_matrix(columns=df.columns[:])
label = labels.as_matrix()

In [4]:
print(data.shape)
print(label.shape)

(124236, 204)
(124236,)


In [5]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.3, random_state=444)

X_TEST, X_VALID, y_TEST, y_VALID = train_test_split(X_test, y_test, test_size = 0.33, random_state=123)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_TEST.shape)
print(y_TEST.shape)

(86965, 204)
(86965,)
(24971, 204)
(24971,)


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(511, activation='relu', input_dim=204))
model.add(Dense(170, activation='relu', input_dim=511))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.compile(optimizer='adagrad',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_VALID, y_VALID))

Train on 86965 samples, validate on 12300 samples
Epoch 1/10
86965/86965 [==============================] - 4s - loss: 0.2042 - acc: 0.9150 - val_loss: 0.1711 - val_acc: 0.9293
Epoch 2/10
86965/86965 [==============================] - 2s - loss: 0.1710 - acc: 0.9288 - val_loss: 0.1717 - val_acc: 0.9272
Epoch 3/10
86965/86965 [==============================] - 2s - loss: 0.1555 - acc: 0.9356 - val_loss: 0.1479 - val_acc: 0.9398
Epoch 4/10
86965/86965 [==============================] - 2s - loss: 0.1425 - acc: 0.9413 - val_loss: 0.1390 - val_acc: 0.9442
Epoch 5/10
86965/86965 [==============================] - 2s - loss: 0.1335 - acc: 0.9457 - val_loss: 0.1428 - val_acc: 0.9393
Epoch 6/10
86965/86965 [==============================] - 2s - loss: 0.1267 - acc: 0.9479 - val_loss: 0.1265 - val_acc: 0.9485
Epoch 7/10
86965/86965 [==============================] - 2s - loss: 0.1210 - acc: 0.9503 - val_loss: 0.1345 - val_acc: 0.9425
Epoch 8/10
86965/86965 [==============================] - 2s 

In [10]:
score = model.evaluate(X_TEST, y_TEST, batch_size=128)

21248/24971 [========================>.....] - ETA: 0s

In [11]:
print(model.metrics_names)
print(score)

['loss', 'acc']
[0.12729456010197573, 0.94737895959312801]


In [12]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [14]:
model.save("predictingMatchesNN.h5")

In [109]:
from sklearn.metrics import confusion_matrix, classification_report

In [115]:
y_test_predict = model.predict(X_TEST, batch_size=128, verbose=1)

23040/24971 [==========================>...] - ETA: 0s

In [116]:
print(y_test_predict, y_test)

[[  1.29029516e-03]
 [  8.77934575e-01]
 [  6.47951439e-02]
 ..., 
 [  4.00680574e-06]
 [  1.83016193e-04]
 [  9.99998212e-01]] [ 1.  0.  1. ...,  0.  1.  0.]


In [117]:
y_prob_to_class = []
for each_value in y_test_predict:
    if each_value[0] < 0.5:
        y_prob_to_class.append(0)
    else:
        y_prob_to_class.append(1)

In [119]:
confusion_matrix(y_prob_to_class, y_TEST)

array([[11903,   669],
       [  618, 11781]])

In [121]:
print(classification_report(y_prob_to_class, y_TEST))

             precision    recall  f1-score   support

          0       0.95      0.95      0.95     12572
          1       0.95      0.95      0.95     12399

avg / total       0.95      0.95      0.95     24971

